In [1]:
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import cv2
import numpy as np
import os
import random
from skimage.transform import swirl

In [2]:
def pop_art(image):
    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(2.5)
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2.0)
    return image

def distortion(image):
    return image.filter(ImageFilter.EMBOSS)

def invert_colors(image):
    return ImageOps.invert(image)

def blur(image):
    return image.filter(ImageFilter.GaussianBlur(radius=5))

def solarize(image, threshold=128):
    return ImageOps.solarize(image, threshold)

def canvas_texture(image):
    img = np.array(image)
    noise = np.random.normal(0, 10, img.shape).astype(np.uint8)
    textured = cv2.add(img, noise)
    return Image.fromarray(textured)

def wave_distortion(image):
    img = np.array(image)
    rows, cols, _ = img.shape
    for i in range(rows):
        img[i] = np.roll(img[i], int(20.0 * np.sin(2 * np.pi * i / 150)), axis=0)
    return Image.fromarray(img)

def mosaic_effect(image, block_size=10):
    img = np.array(image)
    (h, w) = img.shape[:2]
    for y in range(0, h, block_size):
        for x in range(0, w, block_size):
            roi = img[y:y+block_size, x:x+block_size]
            color = roi.mean(axis=(0, 1)).astype(int)
            img[y:y+block_size, x:x+block_size] = color
    return Image.fromarray(img)


PIL_FILTERS = {
    "pop_art": pop_art,
    "distortion": distortion,
    "invert_colors": invert_colors,
    "blur": blur,
    "solarize": solarize,
    "canvas_texture": canvas_texture,
    "wave_distortion": wave_distortion,
    "mosaic_effect": mosaic_effect
}


In [3]:
def swirl_effect(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    swirled = swirl(img, strength=10, radius=300)
    swirled = (swirled * 255).astype(np.uint8)
    return Image.fromarray(swirled)

def pencil_sketch(image_path):
    img = cv2.imread(image_path)
    img_gray, img_sketch = cv2.pencilSketch(img, sigma_s=60, sigma_r=0.07, shade_factor=0.05)
    img_sketch_rgb = cv2.cvtColor(img_sketch, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img_sketch_rgb)

def glitch_effect(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width, _ = img.shape
    glitched = img.copy()
    for _ in range(5):
        x1 = random.randint(0, width//2)
        x2 = random.randint(width//2, width)
        y = random.randint(0, height-5)
        glitched[y:y+5, x1:x2] = np.roll(glitched[y:y+5, x1:x2], 10, axis=1)
    return Image.fromarray(glitched)

def fisheye_effect(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width = img.shape[:2]
    K = np.array([[width, 0, width/2],
                  [0, width, height/2],
                  [0, 0, 1]])
    D = np.array([-0.3, 0.1, 0, 0])
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, (width, height), cv2.CV_16SC2)
    fisheye_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    return Image.fromarray(fisheye_img)

def edge_detection(image_path):
    img = cv2.imread(image_path)
    edges = cv2.Canny(img, 100, 200)
    return Image.fromarray(edges)

def emboss(image_path):
    img = cv2.imread(image_path)
    kernel = np.array([[ -2, -1, 0],
                       [ -1,  1, 1],
                       [  0,  1, 2]])
    embossed = cv2.filter2D(img, -1, kernel) + 128
    return Image.fromarray(cv2.cvtColor(embossed, cv2.COLOR_BGR2RGB))

def clahe_equalization(image_path):
    img = cv2.imread(image_path)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    enhanced = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)
    return Image.fromarray(cv2.cvtColor(enhanced, cv2.COLOR_BGR2RGB))

def oil_painting(image_path):
    img = cv2.imread(image_path)
    oil = cv2.xphoto.oilPainting(img, 7, 1)
    return Image.fromarray(cv2.cvtColor(oil, cv2.COLOR_BGR2RGB))

def stylization_effect(image_path):
    img = cv2.imread(image_path)
    stylized = cv2.stylization(img, sigma_s=60, sigma_r=0.6)
    return Image.fromarray(cv2.cvtColor(stylized, cv2.COLOR_BGR2RGB))

OPENCV_FILTERS = {
    "swirl_effect": swirl_effect,
    "pencil_sketch": pencil_sketch,
    "glitch_effect": glitch_effect,
    "fisheye_effect": fisheye_effect,
    "edge_detection": edge_detection,
    "emboss": emboss,
    "clahe_equalization": clahe_equalization,
    "oil_painting": oil_painting,
    "stylization_effect": stylization_effect
}



In [4]:
# FUNZIONI APPLICAZIONE

def apply_pil_filters(img, filename, output_folder):
    for filter_name, filter_func in PIL_FILTERS.items():
        img_filtered = filter_func(img)
        save_path = os.path.join(output_folder, f"PIL_{filter_name}_{filename}")
        img_filtered.save(save_path)

def apply_opencv_filters(img_path, filename, output_folder):
    for filter_name, filter_func in OPENCV_FILTERS.items():
        img_filtered = filter_func(img_path)
        save_path = os.path.join(output_folder, f"CV_{filter_name}_{filename}")
        img_filtered.save(save_path)

In [5]:
input_folder = "og_pics"
output_folder = "modified_pics"

# Creiamo le cartelle se non esistono
os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Processiamo tutte le immagini nella cartella input
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(input_folder, filename)
        img = Image.open(img_path).convert("RGB")
        
        apply_pil_filters(img, filename, output_folder)
        apply_opencv_filters(img_path, filename, output_folder)

print("Filtri applicati e immagini salvate!")


Filtri applicati e immagini salvate!
